# AnnData to Atlas-Format

Notes: 

- Do not use logarithm-transformed data.

- Use "." as decimal separator.

### Import packages

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
from scipy.sparse import issparse, csr_matrix
from anndata import AnnData
import anndata as ad
import pandas as pd
import gc
import subprocess
import session_info

In [2]:
# Information of the session.

sc.settings.verbosity = 3

session_info.show()

### Insert AnnData object.

In [3]:
# Insert the file path.

adata_file_input = "/Users/rafaelsalgueroraigon/Desktop/archivo_anndata_epithelial_destransformado.h5ad"

In [4]:
# Import AnnData archive.

adata = sc.read_h5ad(adata_file_output)

In [5]:
# Check adata.
adata

AnnData object with n_obs × n_vars = 9691 × 30459
    obs: 'run_id', 'batch', 'relative_position', 'bone', 'scrublet_score', 'doublet_bh_pval', 'anatomical_site', 'pcw', 'brc_code', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'S_score', 'G2M_score', 'phase', 'celltype'
    uns: 'anatomical_site_colors', 'pcw_colors'
    obsm: 'X_umap'
    layers: 'raw_count', 'soupX_corrected'
    obsp: 'connectivities', 'distances'

### Load the function.

If the number of replicates is greater than the number of any cell type, it is not averaged.

In [6]:

def average_replicates(adata, group_key='celltype', n_replicates=5):
    averaged_list = []
    for group_value in adata.obs[group_key].unique():
        mask = adata.obs[group_key] == group_value
        sub_data = adata[mask, :].copy()

        if issparse(sub_data.X):
            sub_data.X = sub_data.X.toarray()

        n_cells = sub_data.shape[0]
        if n_cells < n_replicates:
            averaged_list.append(sub_data)
            continue

        group_size = n_cells // n_replicates
        remainder = n_cells % n_replicates
        group_indices = []
        for i in range(n_replicates):
            size = group_size + 1 if i < remainder else group_size
            group_indices.extend([i] * size)
        group_indices = np.array(group_indices)

        averaged_data = []
        for group_id in range(n_replicates):
            group_mask = group_indices == group_id
            group_mean = np.mean(sub_data.X[group_mask, :], axis=0)
            averaged_data.append(group_mean)

        X = np.round(np.vstack(averaged_data), 2)
        X_sparse = csr_matrix(X)

        averaged_adata = AnnData(
            X=X_sparse,
            var=sub_data.var.copy(),
            obs = pd.DataFrame(index=[f"{group_value}"] * n_replicates)
        )
        averaged_adata.obs[group_key] = group_value
        averaged_list.append(averaged_adata)

    return ad.concat(averaged_list)

1. If the function values are not changed, the default values will be used, with 5 replicates. The optimal number of replicates is between 3 and 10.

2. Group_key indicates the name by which the cell types are identified in the AnnData object.

3. The names of the same types of cell phones must be similar, which is why the warnings appear (they are not important).

In [7]:
average_adata = average_replicates(adata, group_key='celltype', n_replicates=5)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Library/Frameworks/

In [8]:
# Check new adata with averages expression.
average_adata

AnnData object with n_obs × n_vars = 40 × 30459
    obs: 'celltype'

### Get txt archive.

If you don't have enough memory in your computer, use Read_for_chunks = TRUE to read in blocks and convert the contents of the AnnData file to a data frame. Choose chunks size if you use this option.

In [9]:
Read_for_chunks = False
chunk_size = 1000 

In [10]:
if Read_for_chunks:

    chunk_size

    df_list = []

    for start in range(0, average_adata.n_obs, chunk_size):
        end = min(start + chunk_size, average_adata.n_obs)
        block = average_adata.X[start:end]
        block_dense = block.toarray()
        block_df = pd.DataFrame(block_dense, index=average_adata.obs_names[start:end], columns=average_adata.var_names)
        df_list.append(block_df)

    df = pd.concat(df_list)
else:
    
    X = average_adata.X.toarray()
    df = pd.DataFrame(X, index=average_adata.obs_names, columns=average_adata.var_names)

#### Free memory.

In [11]:
del adata
gc.collect()  

867

### Transposition.

(Cells in columns and genes in rows)

##### Define:

- File path

- Block_size.

- By Chunks = True means transposition by chunks. If your computer has enough RAM, By_Chunks = False is recommended.

In [16]:
output_path = "/Users/rafaelsalgueroraigon/Desktop/gene_expresion_transposition.txt"

block_size_transponse = 1000

by_chunks = True  

In [17]:
import pandas as pd

if by_chunks:

    block_size_transponse  
    blocks = []

    for i in range(0, df.shape[0], block_size_transponse):
        block = df.iloc[i:i+block_size_transponse, :]
        blocks.append(block.T) 

    df_T = pd.concat(blocks, axis=1) 

else:
    df_T = df.T  
    
# Save archive in Atlas-Format
df_T.to_csv(output_path, sep='\t', float_format='%.2f')

In [18]:
# Check expression table.

print(df_T.iloc[:10, :10])

           Unknown  Unknown  Unknown  Unknown  Unknown  Periderm  Periderm  \
DAO           0.00     0.00     0.00     0.00     0.00      0.00      0.00   
INTS6-AS1     0.18     0.23     0.16     0.07     0.17      0.05      0.13   
MED11         0.08     0.07     0.10     0.11     0.09      0.10      0.09   
YES1          0.95     0.87     0.88     0.86     0.94      0.58      0.49   
PLEKHG6       0.07     0.05     0.06     0.10     0.04      0.20      0.16   
SLC38A5       0.07     0.06     0.05     0.10     0.07      0.04      0.04   
IQCN          0.03     0.07     0.11     0.14     0.09      0.25      0.14   
CLTRN         0.00     0.01     0.01     0.01     0.00      0.02      0.00   
GUCY1B1       0.28     0.17     0.28     0.17     0.22      0.41      0.33   
ZNF254        1.44     1.12     1.19     1.36     1.38      1.37      1.59   

           Periderm  Periderm  Periderm  
DAO            0.02      0.00      0.00  
INTS6-AS1      0.20      0.09      0.08  
MED11          

### 0 values must have no decimals

Change input_file and output_file path.

In [ ]:
input_file = "/Users/rafaelsalgueroraigon/Desktop/gene_expresion_transposition.txt"
output_file = "/Users/rafaelsalgueroraigon/Desktop/Atlas_Format_Table.txt"

In [19]:
cmd = f"""awk 'BEGIN{{FS=OFS="\\t"}} {{for(i=1;i<=NF;i++) if($i=="0.00") $i="0"; print}}' \
"{input_file}" > "{output_file}" """

subprocess.run(cmd, shell=True)

CompletedProcess(args='awk \'BEGIN{FS=OFS="\\t"} {for(i=1;i<=NF;i++) if($i=="0.00") $i="0"; print}\' "/Users/rafaelsalgueroraigon/Desktop/gene_expresion_transposition.txt" > "/Users/rafaelsalgueroraigon/Desktop/Atlas_Format_Table.txt" ', returncode=0)

# Reference example data: 

To, K., Fei, L., Pett, J.P. et al. A multi-omic atlas of human embryonic skeletal development. Nature 635, 657–667 (2024). https://doi.org/10.1038/s41586-024-08189-z

# Original autors:

Rafael Salguero Raigón

Iván Delgado Sánchez

Javier Santos del Río